# Cellulaire automaat in Python

Deze versie is bedoeld voor de microbit; maar we kunnen ook een versie maken voor Jupyter Notebook cq. Jupyter Book.

(Voor de JB versie is het netter cq. handiger om een groter veld te gebruiken. (Eigenlijk heeft alleen de huidige generatie daar betekenis; maar we kunnen wel een (beperkte) geschiedenis bijhouden; kunnen we daar het display van afleiden?) In de onderstaande versie moeten we de grootte van het veld instelbaar maken.

(Wat hebben we nodig voor de grens-cellen? - we kunnen daar ook een functie van maken: dan kunnen we altijd nog kiezen of we "rond willen tellen", of met een 0-grens werken; of zelfs met een 1-grens...)

In [32]:
import random
from typing import List

rule = [0, 0, 0, 0, 0, 0, 0, 0]  # rule represented as 8 bits

size = 10     # number of cells in a state
state = []    # a list of cells
history = []  # a list of states

# create a new cellular automaton
# nr_cells: number of cells
# ones:     positions of living cells
def create_state(nr_cells: int, ones: List[ int ]) -> None:
    global size, state, history
    size = nr_cells
    state = []
    for i in range(size):
        if i in ones:
            state.append(1)
        else:
            state.append(0)
    history = [state]

# set rule-list from rule-number
def set_rule(rulenr: int) -> None:
    global rule
    for r in range(8):
        rule[r] = rulenr % 2
        rulenr = rulenr // 2

# cell-value in current state,
# with border-cells with value 0
def cell(i: int) -> int:
    if i < 0 or i >= len(state):
        return 0
    else:
        return state[i]

# compute next state
# and add to history
def step_state() -> None:
    global state, history
    next_state = []
    for i in range(size):
        cell_state = cell(i-1) * 4 + cell(i) * 2 + cell(i+1)
        next_state.append(rule[cell_state]) 
    state = next_state
    history.append(state)

In [36]:
def display_history() -> None:
    for row in history:
        line = ""
        for x in row:
            line = line + str(x)     
        print(line)         

In [37]:
create_state(40, [20,35])
set_rule(110)
# new_seed()


In [38]:
display_history()

0000000000000000000010000000000000010000


In [39]:
print(rule)

[0, 1, 1, 1, 0, 1, 1, 0]


In [40]:
step_state()
display_history()

0000000000000000000010000000000000010000
0000000000000000000110000000000000110000


In [41]:
for i in range(20):
    step_state()
display_history()    

0000000000000000000010000000000000010000
0000000000000000000110000000000000110000
0000000000000000001110000000000001110000
0000000000000000011010000000000011010000
0000000000000000111110000000000111110000
0000000000000001100010000000001100010000
0000000000000011100110000000011100110000
0000000000000110101110000000110101110000
0000000000001111111010000001111111010000
0000000000011000001110000011000001110000
0000000000111000011010000111000011010000
0000000001101000111110001101000111110000
0000000011111001100010011111001100010000
0000000110001011100110110001011100110000
0000001110011110101111110011110101110000
0000011010110011111000010110011111010000
0000111111110110001000111110110001110000
0001100000011110011001100011110011010000
0011100000110010111011100110010111110000
0110100001110111101110101110111100010000
1111100011011100111011111011100100110000
1000100111110101101110001110101101110000


Hoe passen we dit aan voor de microbit?

* bij het maken van de automaat beperken we ons tot size = 5.
* we tonen op het display alleen de laatste 5 generaties
* we hoeven dan in de history ook niet meer dan 5 generaties op te slaan.
    * (dat spaart op de microbit weer wat schaars geheugen)
    * (eigenlijk moeten we dan wel zo nu en dan garbage collection doen?)

Voor het zetten van de initiële toestand kunnen we knop B gebruiken als een binaire teller. (Zie andere opdrachten.)


In [24]:
create_state(5, [4])
set_rule(110)

In [25]:
def display_history_mb():
    # display.clear()
    if len(history) < 5:
        for row in history:
            line = ""
            for cell in row:
                line = line + str(cell)
            print(line)
    else:
        for row in history[-5:]:
            line = ""
            for cell in row:
                line = line + str(cell)
            print(line)

In plaats van `print(line)` wordt dit:

```Python
for colnr in range(5):
    display.set_pixel(rownr, colnr, cell * 9)
rownr = rownr + 1    
```

Bovendien hoeven we niet de hele historie te onthouden.
In plaats van history.append(state) krijgen we dan:

```Python
   history.append(state)
   if len(history) > 5:
       history.pop(0)
```

In [26]:
display_history_mb()

00001


In [31]:
for i in range(5):
    step_state()
    if len(history) > 5:
       history.pop(0)
    display_history()
    print("-")

..xxx
.xx.x
xxxxx
x...x
...xx
-
.xx.x
xxxxx
x...x
...xx
..xxx
-
xxxxx
x...x
...xx
..xxx
.xx.x
-
x...x
...xx
..xxx
.xx.x
xxxxx
-
...xx
..xxx
.xx.x
xxxxx
x...x
-


Voor het gebruik van 

In [28]:
def inc_state():
    statenr = 0
    for cell in state:
        statenr = statenr * 2 + cell
    statenr = statenr + 1
    for i in range(size):
        state[-i-1] = statenr % 2
        statenr = statenr // 2

In [29]:
print(state)
state = [0, 0, 0, 0, 0]
print(state)

[1, 0, 0, 0, 1]
[0, 0, 0, 0, 0]


In [30]:
inc_state()
print(state)

[0, 0, 0, 0, 1]


De constructie om een list (array)-waarde te maken `size` 0-en kun je in Python ook als volgt schrijven:

```Python
[0 for i in range(size)]
```

Dit is een voorbeeld van *list comprehension*.

In [15]:
[0 for i in range(size)]

[0, 0, 0, 0, 0]